# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [91]:
# Вызовем библиотеку pandas, импортируем и откроем датасет:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**


В данных есть пропуски. В столбцах days_employed и total_income одинкаовое колличество пропусков. 
В days_employed пропуски можно заполнить простым средним значением, так как нам эти данные не нужны.
В total_income следует заменить пропуски с привязкой к другим столбцам, чтобы получать более корректные данные.


## Шаг 2. Предобработка данных

### Обработка пропусков

In [92]:
# взглянем на строки с пропусками:
df[df['days_employed'].isna()].head(20)

# заменим пропуски в days_employed:

df['days_employed'] = df['days_employed'].fillna(value=df['days_employed'].mean()) 


# заменим пропуски в total_income:

 
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda s: s.fillna(s.mean()))

# проверим данные:

df.info()

df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [93]:
# Создам датафрейм.

l = {'A':[1,2,4], 'B':[3,3,5]}

dfq = pd.DataFrame(l)
dfq

,A,B
0,1,3
1,2,3
2,4,5


In [94]:
# Обычная функция. 
def squared(df):
    ''' 
    Функция принимает таблицу, меняет элементы на их квадраты и возвращает уже таблицу с квадратами.
    '''
    df['A'] = df['A']*df['A']
    df['B'] = df['B']*df['B']
    return df

squared(dfq)

,A,B
0,1,9
1,4,9
2,16,25


In [95]:

# Таблица изменилась, потому функция ее вернула измененную.
dfq

,A,B
0,1,9
1,4,9
2,16,25


In [96]:
def unsquared(df):
    ''' 
    Берет квадратный корень. 
    
    Принимает: таблицу.
    Возвращает: таблицу, из элементов которой взят квадратный корень.
    '''
    
    import numpy as np
    
    df['A'] = np.sqrt(df['A']).astype(int)
    df['B'] = np.sqrt(df['B']).astype(int)
    return df

# Вернем все назад.
unsquared(dfq)

,A,B
0,1,3
1,2,3
2,4,5


In [97]:

# Таблица вернулась, потому функция ее вернула измененную.
dfq

,A,B
0,1,3
1,2,3
2,4,5


In [98]:
def sum_(row):
    ''' 
    Функция принимает таблицу построчно. Возвращает сумму элементов. 
    Применяется ко всей таблице через метод apply с аргументом axis=1, то есть построчно.
    '''
    return row['A']+row['B']

dfq['Sum'] = dfq.apply(sum_, axis=1)
dfq

,A,B,Sum
0,1,3,4
1,2,3,5
2,4,5,9


In [99]:

dfq['Sum2'] = dfq.apply(lambda row: row['A'] + row['B'], axis=1)
dfq

,A,B,Sum,Sum2
0,1,3,4,4
1,2,3,5,5
2,4,5,9,9


In [100]:

# Сумма по всей строке может быть взята проще:
dfq['Total_sum'] = dfq.apply(lambda row: row.sum(), axis=1)
dfq

,A,B,Sum,Sum2,Total_sum
0,1,3,4,4,12
1,2,3,5,5,15
2,4,5,9,9,27


In [101]:

import numpy as np

# Сумма столбца. Добавим строку.
dfq.loc[3] = dfq.apply(np.sum, axis=0)
dfq

,A,B,Sum,Sum2,Total_sum
0,1,3,4,4,12
1,2,3,5,5,15
2,4,5,9,9,27
3,7,11,18,18,54


In [103]:
df1 = df.copy()
df1.education = df1.education.str.lower()

df1.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')

education                      высшее      начальное  неоконченное высшее  \
income_type     gender                                                      
безработный     F       202722.511368            NaN                  NaN   
                M                 NaN            NaN                  NaN   
в декрете       F                 NaN            NaN                  NaN   
госслужащий     F       170622.718045  138332.356612        154277.551439   
                M       204717.884325  190966.659534        170898.309923   
компаньон       F       200981.752861  152336.943284        180705.451982   
                M       217768.009290  150100.960964        200559.696452   
                XNA               NaN            NaN        203905.157261   
пенсионер       F       137127.465690  102598.653164        130837.388717   
                M       150246.754511  114068.787524        124667.471301   
предприниматель F       499163.144947            NaN                  NaN   
                M       499163.144947            NaN                  NaN   
сотрудник       F       161380.260488  124397.467932        143663.050521   
                M       179564.658485  147989.228695        175068.921286   
студент         M        98201.625314            NaN                  NaN   

education                     среднее  ученая степень  
income_type     gender                                 
безработный     F                 NaN             NaN  
                M        59956.991984             NaN  
в декрете       F        53829.130729             NaN  
госслужащий     F       135073.137857   111392.231107  
                M       170898.309923             NaN  
компаньон       F       155725.928731             NaN  
                M       191468.957958             NaN  
                XNA               NaN             NaN  
пенсионер       F       123508.906819   255425.196556  
                M       134708.033691    98752.495442  
предприниматель F                 NaN             NaN  
                M                 NaN             NaN  
сотрудник       F       132635.207938             NaN  
                M       161380.260488   198570.757322  
студент         M                 NaN             NaN

In [104]:

qq = df1.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')
qq.loc[('студент', 'M')]['высшее']

98201.62531401133

In [105]:

def super_fillna_func(income_type, gender, education):
    '''
    Находит в таблице qq нужную медиану.
    '''
    try:
        return qq.loc[(income_type, gender)][education]
    except:
        return 0
    
print(super_fillna_func('студент', 'M','высшее') ) 

98201.62531401133


In [106]:

print(super_fillna_func('ревьюер', 'F','высшее') )

0


In [107]:

# Так сработает apply.
df1.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1)

0        161380.260488
1        132635.207938
2        161380.260488
3        161380.260488
4        123508.906819
             ...      
21520    155725.928731
21521    123508.906819
21522    161380.260488
21523    161380.260488
21524    132635.207938
Length: 21525, dtype: float64

In [108]:

# Вот так применяем к таблице.

# Запишем в новый столбец.
# df1['new_income'] = df1.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1)

# Пандас сам заменит пропуски значениями из нового столбца в той же строке.
# df1['total_income'] = df1['total_income'].fillna(df['new_income'])

In [109]:

# А вот так мы столбец создадим, но сохранять не будем. Замену произведем в воздухе:
df1['total_income'] = df1['total_income'].fillna(df1.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1))


    
Ну и то же самое, но через лямбду и трансформ: 

In [110]:

# df1['total_income'] = df1.groupby(['income_type','gender', 'education'])['total_income'].apply(lambda x: x.fillna(x.median()))
   
# Оно же, но без лямбды:
# df1['total_income'] = df1['total_income'].fillna(df1.groupby(['income_type','gender', 'education'])['total_income'].transform('median'))

**Вывод**

Пропуски исправлены, можно двигаться дальше. Скорее всего пропуски были техническим сбоем, так как пропуски были в весьма значимых категориях данных, без которых, скорее всего, в кредитовании было бы отказано.

### Замена типа данных

In [111]:
# переведем вещественные значения в целочисленные:

df['total_income'] = df['total_income'].astype('int')

# проверим успешность операции:

df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Нам предстоить работать со столбцом total_income, поэтому его значения есть смысл перевести в Int64. В трансформации столбца days_employed нет необходимости. Был использован метод .astype(), поскольку метод .to_numeric() возвращает только вещественные значения.

### Обработка дубликатов

In [112]:
# найдем явные дубликаты:

df.duplicated().sum()

# приведем все к нижнему регистру:

df['education'] = df['education'].str.lower()

# проверим:

display(df.head(10))

# удалим дубликаты:

df = df.drop_duplicates()

# приверим еще раз наличие дубликатов:

df.duplicated().sum()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


0

**Вывод**

Был найден 71 явный дубликат, который мы в удалили методом .drop_duplicates(). 

### Лемматизация

In [113]:
# импортируем библиотеку для лемматизации:
from pymystem3 import Mystem
m = Mystem() 

# Создаем функцию, которая принимает столбец на вход

lemmas = []
def lem_purpose(column): 
    lemmas = m.lemmatize(column) # записываем результат лемматизации столбца в переменниую
    return lemmas # возвращаем переменную
 
df['purpose_lemmas'] = df['purpose'].apply(lem_purpose) # применяем и записываем в новый столбец 'purpose_lemmas'
 
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


**Вывод**

Здесь, я, пожалуй, не совсем понял что требуется. Выделил леммы для целей кредита и создал с ними отдельный столбец, к которму буду обращаться в исследовании

### Категоризация данных

In [114]:
# функция для категоризации целей кредита:

def purpose_category(purpose_lemmas):
    if (('жилье' in purpose_lemmas) or ('недвижимость' in purpose_lemmas)) and ('строительство' not in purpose_lemmas):
        return 'покупка недвижимости'
    if (('жилье' in purpose_lemmas) or ('недвижимость' in purpose_lemmas)) and ('строительство' in purpose_lemmas):
        return 'строительство недвижимости'
    if 'автомобиль' in purpose_lemmas:
        return 'покупка автомобиля'
    if 'образование' in purpose_lemmas:
        return 'образование'
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'
    return purpose_lemmas

df['purpose_category'] = df['purpose_lemmas'].apply(purpose_category)
df['purpose_category'].value_counts()

# функция для категоризации уровня дохода:

def total_income_category(total_income):
    if total_income <= 100000:
        return 'низкий достаток'
    if 100000 < total_income <= 150000:
        return 'средний достаток'
    if 150000 < total_income <= 200000:
        return 'высокий достаток'
    if total_income > 200000:
        return 'богатые'
    
df['total_income_category'] = df['total_income'].apply(total_income_category)
df['total_income_category'].value_counts()

средний достаток    6090
богатые             5568
высокий достаток    5333
низкий достаток     4463
Name: total_income_category, dtype: int64

**Вывод**

Категории целей, на мой взгляд, выделены правильно. 
Категоризацию доходов я выделил в группы с шагом в 50000 рублей, исходя из своего личного опыта, поскольку прибегая к данным из росстата, человеку необходимо около 12000 рублей в месяц. На мой взгляд это некорректные данные. Тем более, что распределение по группам получилось почти соразмерным. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [115]:
#создаем сводную таблицу для анализа соотношения количества детей к вероятности возврата кредита
df_pivot_children = df.pivot_table(index=['children'], columns=['debt'], values=['total_income'], aggfunc='count')

# смотрим на результат:
print(df_pivot_children.head(10))

# считаем зависимость количества детей от успешности возврата кредита:
df_pivot_children_ratio = df_pivot_children['total_income'][1] / df_pivot_children['total_income'][0] * 100
df_pivot_children_ratio.head(10)


         total_income        
debt                0       1
children                     
-1               46.0     1.0
 0            13028.0  1063.0
 1             4364.0   444.0
 2             1858.0   194.0
 3              303.0    27.0
 4               37.0     4.0
 5                9.0     NaN
 20              68.0     8.0


children
-1      2.173913
 0      8.159349
 1     10.174152
 2     10.441335
 3      8.910891
 4     10.810811
 5           NaN
 20    11.764706
dtype: float64

**Вывод**

Данными с количеством детей -1, 5 и 20 считаю можно принебречь, поскольку таких результатов менее одного процента. Из итоговой табицы видно, что процент должников без детей - 8.15 %, с 1,2 и 4 детьми - 10-11 %. Таким образом заемщики с детьми невозвращают кредит чаще, чем заемщики без детей. Однако эта разница составляет 2-2.5%. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [116]:
#создаем сводную таблицу для анализа соотношения семейного положения к вероятности возврата кредита
df_pivot_family = df.pivot_table(index=['family_status'], columns=['debt'], values=['total_income'], aggfunc='count')

# смотрим на результат:
print(df_pivot_family.head(10))

# считаем зависимость семейного положения от успешности возврата кредита:
df_pivot_family_ratio = df_pivot_family['total_income'][1] / (df_pivot_family['total_income'][0] + df_pivot_family['total_income'][1]) * 100
df_pivot_family_ratio.head()

                      total_income     
debt                             0    1
family_status                          
Не женат / не замужем         2536  274
в разводе                     1110   85
вдовец / вдова                 896   63
гражданский брак              3763  388
женат / замужем              11408  931


family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
dtype: float64

In [117]:
# КОД РЕВЬЮЕРА

df.groupby('family_status')['debt'].agg(['count','sum','mean'])

,count,sum,mean
family_status,,,
Не женат / не замужем,2810,274,0.097509
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452


In [118]:
# КОД РЕВЬЮЕРА

# В кач-ве агрегирования можно любую функцию брать.
df.groupby('family_status')['debt'].agg(['count', 'sum', lambda x: x.mean()*100])

,count,sum,<lambda_0>
family_status,,,
Не женат / не замужем,2810,274,9.750890
в разводе,1195,85,7.112971
вдовец / вдова,959,63,6.569343
гражданский брак,4151,388,9.347145
женат / замужем,12339,931,7.545182


In [119]:
# КОД РЕВЬЮЕРА

# Или просто форматируем вывод
df.groupby('family_status')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
family_status,,,
Не женат / не замужем,2810,274,9.75%
в разводе,1195,85,7.11%
вдовец / вдова,959,63,6.57%
гражданский брак,4151,388,9.35%
женат / замужем,12339,931,7.55%


**Вывод**

Средний процент должников категорий "в разводе", "вдовец/вдова", "женат/замужем" = 7%
Средний процент должников категорий "Не женат/не замужем", "гражданский брак" = 9.5%
Таким образом, должников, берущих заемные средства до заключения официального брака больше на 2.5%, чем должников после регистрации брака. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [120]:
#создаем сводную таблицу для анализа соотношения дохода к вероятности возврата кредита
df_pivot_income = df.pivot_table(index=['total_income_category'], columns=['debt'], values=['total_income'], aggfunc='count')

# смотрим на результат:
print(df_pivot_income.head(10))

# считаем зависимость дохода от успешности возврата кредита:
df_pivot_income_ratio = df_pivot_income['total_income'][1] / (df_pivot_income['total_income'][0] + df_pivot_income['total_income'][1]) * 100
df_pivot_income_ratio.head()

                      total_income     
debt                             0    1
total_income_category                  
богатые                       5180  388
высокий достаток              4860  473
низкий достаток               4109  354
средний достаток              5564  526


total_income_category
богатые             6.968391
высокий достаток    8.869304
низкий достаток     7.931884
средний достаток    8.637110
dtype: float64

**Вывод**

Богатые и бедные возвращают кредит на 1-2% чаще, чем люди со средними зарплатами.

- Как разные цели кредита влияют на его возврат в срок?

In [121]:
#создаем сводную таблицу для анализа соотношения целей кредита к вероятности возврата кредита
df_pivot_target = df.pivot_table(index=['purpose_category'], columns=['debt'], values=['total_income'], aggfunc='count')

# смотрим на результат:
print(df_pivot_target.head(10))

# считаем зависимость целей кредита от успешности возврата кредита:
df_pivot_target_ratio = df_pivot_target['total_income'][1] / (df_pivot_target['total_income'][0] + df_pivot_target['total_income'][1]) * 100
df_pivot_target_ratio.head()

                           total_income     
debt                                  0    1
purpose_category                            
образование                        3643  370
покупка автомобиля                 3903  403
покупка недвижимости               8295  638
свадьба                            2138  186
строительство недвижимости         1734  144


purpose_category
образование                   9.220035
покупка автомобиля            9.359034
покупка недвижимости          7.142058
свадьба                       8.003442
строительство недвижимости    7.667732
dtype: float64

**Вывод**

Должников с целями на образование или автомобиль больше на 1-2%, чем должников с целями на недвижимость или свадьбу.

## Шаг 4. Общий вывод

В исследовании выявлено, что средняя разница возврата и невозврата кредита в разных группах граждан составляет в среднем от 1.5 до 3%. 
Самые надежные заемщики после заключения брака - 7% невозврата в срок. 
Самые злостные неплательщики - до брака - 9% невернувших деньги.
Самые надежные заемщики богатые, с доходом от 200 000 рублей в месяц  - 7% невернувших кредит.
Самые злостные неплательщики - люди со средним достатком от 100 000 до 200 000 - 8.5% невернувших кредит.
Самые надежные заемщики - покупатели недвижимости - 7% должников.
Самые злостные неплательщики - автомобилисты - 9.4% должников.
На мой взгляд, разница не большая, но в рамках большого объема выдачи кредитов, убытки могут быть существенны. Если кредитная организация сделает выводы, и пересмотрит алгоритмы выдачи кредитов, пожалуй она (кредитная организация) сможет повысить свою доходность. В анализе я осознанно выводил на экран сначала количество погашенных и непогашенных кредитов, а потом их соотношение, поскольку 1% от 100 и 1% 10000 это немного разные цифры, и эти данные также стоит учитывать. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.